## Projet Greedy Algorithm, NAVARRE Victor CACHERA Alexandre

In [ ]:
#Importation des librairies

import numpy as np

On commence par décrire l'Algorithme Greedy faible pour les n-withd, on réalise une implémentation récursive

In [ ]:
def projection_orthogonale(f, Vn):

    V = np.column_stack(Vn)  # Colonne = vecteur de base

    coeffs, _, _, _ = np.linalg.lstsq(V, f, rcond=None)
    proj = V @ coeffs
    return proj

def dist_to_span(f,Vn):
    return(np.linalg.norm(f-projection_orthogonale(f,Vn)))


In [6]:
def weakgreedy (A,lambd,n):
    Vn=[]
    Vn.append(max(np.linalg.norm(v) for v in A))             #On rajoute f0
    Sigma_i=(max(dist(f,Vn) for f in A))
    for i in range(n):
        for f in range(len(A)):
            if dist(f,Vn)>=lambd*Sigma_i:
                Vn.append(f)
                A.remove(f)
                break
    Sigma_n=(max(dist(f,Vn) for f in A))
    return("Espace vectoriel engendrée",Vn,"Distance obtenue",Sigma_n )


Algorithme de PCA et autres algorithmes

Différentes données

Analyse des résultats

Conclusion
